In [1]:
import psycopg2
import pandas as pd
import numpy as np
from psycopg2 import sql
from datetime import datetime, timedelta

# 1. Подключение к PostgreSQL

Создаем подключение к таблице через psycopg2

In [2]:
dbname = 'People'
user = 'postgres'
password = '1234'
host = 'localhost'


try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
except:
    print('Can`t establish connection to database')

# 2. Запросы из бд

Пробуем сделать запрос из бд после подключения. Для этого используем конструкцию `with`, чтобы закрывать объект cursor сразу после запроса:

In [3]:
cur = conn.cursor()

with cur as curs:
    curs.execute('SELECT * FROM users')
    # Получение результатов запроса в виде списка кортежей
    all_users = curs.fetchall()

# 3. Получаем данные в виде pandas Dataframe

In [4]:
df = pd.DataFrame(all_users, columns=[desc[0] for desc in cur.description])

# 4. Добавление новых таблиц в базу данных

Генерируем новые данные для дальнешей загрузки их в нашу базу данных:

In [5]:
install_list = []
install_time = datetime.strptime("2024-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")

for i in range(1, 501):
    app_name = f'name_{np.random.randint(1, 6)}'
    install_time = install_time + timedelta(hours=np.random.randint(1, 25),
                                            seconds=np.random.randint(1, 3601)
                                           )
    
    install_list.append({'id': i, 'app_name': app_name, 'install_time': install_time})

# Создаем фрейм данных для активности
installs = pd.DataFrame(install_list).sort_values('install_time')

Создаем запрос для создания таблицы в нашей бд, затем с помощью `with` осуществляем запрос

In [6]:
create_table_query = '''
CREATE TABLE installs (
    id SERIAL PRIMARY KEY,
    app_name VARCHAR(255),
    install_time TIMESTAMP
)
'''

cur = conn.cursor()

with cur as curs:
    curs.execute(create_table_query)
    # Получение результатов запроса в виде списка кортежей
    conn.commit()

Затем создаем запрос, с помощью которого мы сможем добавлять наши данные в таблицу:

In [7]:
table_name = 'installs'
query = (
    sql.SQL('''
        INSERT INTO {} (id, app_name, install_time)
        VALUES (%s, %s, %s)
        ''').format(sql.Identifier(table_name))
)

values = [tuple(x) for x in installs.values]

cur = conn.cursor()

with cur as curs:
    curs.executemany(query, values)
    # Получение результатов запроса в виде списка кортежей
    conn.commit()

Генерируем таблицу `levels` с условиями, что дата активности нашего пользователя должна быть больше, чем дата регистрации в приложении(`installs`), а также каждая последующая дата обновления уровня выше, чем предыдущая дата получения уровня, которая связана вторичным ключом с нашей таблицей `installs`

In [8]:
activities = []

for index, row in installs.iterrows():
    num_levels = np.random.randint(1, 41)
    event_time = row['install_time']
    
    for level in range(1, num_levels + 1):
        event_time = event_time + timedelta(hours=np.random.randint(1, 25),
                                            seconds=np.random.randint(1, 3601)
                                           )
        activities.append({'user_id': row['id'], 'level': level, 'event_time': event_time})

# Создаем фрейм данных для активности
levels = pd.DataFrame(activities).sort_values('event_time')

In [9]:
create_table_query = '''
CREATE TABLE levels (
    user_id INT REFERENCES installs(id),
    level INT,
    event_time TIMESTAMP
)
'''

cur = conn.cursor()

with cur as curs:
    curs.execute(create_table_query)
    # Получение результатов запроса в виде списка кортежей
    conn.commit()

In [10]:
table_name = 'levels'
query = (
    sql.SQL('''
    INSERT INTO {} (user_id, level, event_time)
    VALUES (%s, %s, %s)
    ''').format(sql.Identifier(table_name))
)

values = [tuple(x) for x in levels.values]

cur = conn.cursor()

with cur as curs:
    curs.executemany(query, values)
    # Получение результатов запроса в виде списка кортежей
    conn.commit()